In [1]:
import json
import requests

In [2]:
from langchain.tools import OpenAPISpec
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

In [3]:
"""
First, register your agent!
Name of the faction (preferably "COSMIC")
Choose a symbol for your agent
Add your email to map the agent to your user
"""
faction_info = {
    "faction": "",
    "symbol": "",
    "email": ""
}
register_agent = requests.post("https://api.spacetraders.io/v2/register", data=faction_info)
spacetraders_auth_token = register_agent.json()['data']['token']

In [4]:
# load the OpenAPI spec
raw_spacetraders_spec = OpenAPISpec.from_file('SpaceTrader_LLM_Documentation/SpaceTraders-OpenAPI.json')
converted_spacetraders_spec = json.loads(raw_spacetraders_spec.json())
reduced_spacetraders_spec = reduce_openapi_spec(converted_spacetraders_spec)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [5]:
from langchain.chat_models import ChatOpenAI

In [6]:
from langchain.agents.agent_toolkits.openapi import planner

In [7]:
# use your openai api key here
llm = ChatOpenAI(openai_api_key='', model_name='gpt-3.5-turbo', temperature=0.0)

In [8]:
def spacetraders_auth():
    auth_token = spacetraders_auth_token
    return {'Authorization': "Bearer " + auth_token}

In [9]:
from langchain.requests import RequestsWrapper
spacetraders_headers = spacetraders_auth()
spacetraders_requests_wrapper = RequestsWrapper(headers=spacetraders_headers)

In [10]:
# create the SpaceTraders game playing agent
spacetraders_agent = planner.create_openapi_agent(api_spec=reduced_spacetraders_spec, requests_wrapper=spacetraders_requests_wrapper, llm=llm)

In [20]:
user_query = ("You are agent " + faction_info["symbol"] + " who is a SpaceTrader. You control a fleet of ships. Your goal is to earn credits and reputation through the successful completion of contracts. Complete the following steps: " +
              "1. View the outstanding contracts " +
              "2. Accept the outstanding contract" +
              "3. Find and go to nearest shipyard and purchase a mining drone. " +
              "4. Ensure you are undocked and in-orbit " +
              "5. Navigate mining drone to nearby asteroid field " +
              "6. Once you reach the asteroid field, dock your ship using the appropriate POST request " +
              "7. extract ores and minerals. " +
              "8. navigate to destination specified in the contract " +
              "9. Deliver contract goods at the destination" +
              "10. Fulfill contract.")

In [19]:
spacetraders_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: "View the outstanding contracts"
Observation: 1. GET /my/contracts to retrieve a paginated list of all the user's contracts.
Thought:I have the plan for the first step. Now I need to generate a plan for the second step.
Action: api_planner
Action Input: "Accept the outstanding contract"
Observation: 1. GET /my/contracts to retrieve a list of all your contracts.
2. Find the outstanding contract that has not been accepted yet.
3. POST /my/contracts/{contractId}/accept to accept the contract.
Thought:I have the plan for the second step. Now I need to generate a plan for the third step.
Action: api_planner
Action Input: "Find and go to nearest shipyard and purchase a mining drone"
Observation: 1. GET /my/agent to fetch the agent's details.
2. GET /my/ships to fetch a paginated list of all ships under the agent's ownership.
3. POST /my/ships/{shipSymbol}/scan/waypoints to scan for nearby waypoints and retrieve detaile

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 8875 tokens. Please reduce the length of the messages.